# **Aplicando OCR - Usando EasyOCR**

## Iniciando com EasyOCR

READ the Documentation: https://www.jaided.ai/easyocr/documentation/

Instale a biblioteca EasyOCR no seu ambiente Python:

In [ ]:
!pip install easyocr

Importe a biblioteca EasyOCR e OpenCV. Também importe outras bibliotecas que você achar necessário

In [ ]:
import easyocr
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow 

Carregue a imagem que você deseja reconhecer o texto usando openCV e execute a função readtext() da biblioteca EasyOCR:

In [ ]:
# Carrega a imagem utilizando o OpenCV
imagem = cv2.imread('/content/geralt.jpg')
cv2_imshow(imagem)

Agora instancie um objeto da classe Reader. Esse objeto será o responsável por identificar os textos na imagem

In [ ]:
# Criando o Reader para o idioma Inglês (english) e dizendo para usar GPU
reader = easyocr.Reader(lang_list=['en'], gpu = True)

Carregue a imagem que você deseja reconhecer o texto e execute a função readtext() da biblioteca EasyOCR:

In [ ]:
texto = reader.readtext(image = imagem, detail = 0)


O resultado é uma lista com cada linha "lida" da imagem

In [ ]:
texto

Percebe-se que o resultado é até bom, mas temos alguns erros, como por exemplo:

* No treho "Lesser, greater, middling...", ele subistitui a reticências por hifen;

* No trecho "If I'm to choose...", ele substituiu o I' por T;

* O mesmo ocorre na ultima linha, com "I'd rather...". 

Vamos ignorar esses erros por enquanto...

## Lendo Textos Completos

 Agora vamos usar uma outra imagem de um texto mais estruturado

In [ ]:
imagem2 = cv2.imread('/content/dissertacao.JPG')
cv2_imshow(imagem2)

In [ ]:
texto = reader.readtext(image = imagem2, detail = 0)
texto

Neste caso, cada linha foi colocada num item da lista. Para que o texto seja lido como um paragrafo, precisamos usar outro parametro, o paragraph!

In [ ]:
texto = reader.readtext(image = imagem2, detail = 0, paragraph = True)
texto

## Identificando a posição do texto na imagem

In [ ]:
imagem3 = cv2.imread('/content/avenida.JPG')
cv2_imshow(imagem3)

Usando o Reader do EeasyOCR, temos mais informações além do texto em si. O Reader nos retorna também uma lista de tuplas, em que além do texto, ele também nos informa em que parte da imagem ele identificou aquele texto e qual o grau de confiança que ele tem.

In [ ]:
resultado = reader.readtext(image = imagem3)
resultado

Com essas informações, podemos desenhar um retangulo ao redor das palavras detectadas. Para isso, precisamos das coordenadas de cada vertice do retangulo para usarmos o método cv2.retangle, que precisa dos seguintes parametros:

* imagem onde será desenhado o retangulo;

* ponto do vertice superior esquerdo;

* ponto do vertice inferior direito;

* cor do retangulo no formato BGR;

* espessura da linha do retangulo;

In [ ]:
for(coordenadas, texto, probabilidade) in resultado:
  if(probabilidade >= 0.5):
    (sup_esq, sup_dir, inf_dir, inf_esq) = coordenadas
    sup_esq = (int(sup_esq[0]),int(sup_esq[1]))
    sup_dir = (int(sup_dir[0]),int(sup_dir[1]))
    inf_dir = (int(inf_dir[0]),int(inf_dir[1]))
    inf_esq = (int(inf_esq[0]),int(inf_esq[1]))
    cv2.rectangle(imagem3,sup_esq, inf_dir,(255,0,0),2)
cv2_imshow(imagem3)

In [ ]:
imagem4 = cv2.imread('/content/avenida2.JPG')
resultado = reader.readtext(image = imagem4)
for(coordenadas, texto, probabilidade) in resultado:
  if(probabilidade >= 0.3):
    (sup_esq, sup_dir, inf_dir, inf_esq) = coordenadas
    sup_esq = (int(sup_esq[0]),int(sup_esq[1]))
    sup_dir = (int(sup_dir[0]),int(sup_dir[1]))
    inf_dir = (int(inf_dir[0]),int(inf_dir[1]))
    inf_esq = (int(inf_esq[0]),int(inf_esq[1]))
    cv2.rectangle(imagem4,sup_esq, inf_dir,(255,0,0),2)
cv2_imshow(imagem4)

In [ ]:
resultado

## Aplicações de OCR

Vamos agora fazer algumas aplicações básicas:

1 - Integrar o OCR com uma biblioteca Text to Speech para gerar um audio book, para isso, precisamos instalar a biblioteca gtts (Google Text To Speech) e a biblioteca mpg321, para salvar o arquivo mp3

In [ ]:
!pip install gtts
!apt-get install -y mpg321

In [ ]:
from gtts import gTTS

# Carregando a imagem e criando o texto via OCR
imagem = cv2.imread('/content/geralt.jpg')
reader = easyocr.Reader(lang_list=['en'], gpu = True)
texto = ' '.join(reader.readtext(image = imagem, detail = 0))

# Cria um objeto gTTS com o texto reconhecido
tts = gTTS(text=texto, lang='en')

# Salva o arquivo de áudio em disco
tts.save('GeraldQuote.mp3')

agora com um texto completo

In [ ]:
# Carregando a imagem e criando o texto via OCR
imagem = cv2.imread('/content/dissertacao.JPG')
reader = easyocr.Reader(lang_list=['en'], gpu = True)
texto = ' '.join(reader.readtext(image = imagem, detail = 0))

# Cria um objeto gTTS com o texto reconhecido
tts = gTTS(text=texto, lang='en')

# Salva o arquivo de áudio em disco
tts.save('Dissertacao.mp3')

2 - Um tradutor que pode ser usado apenas apontando a camera para uma palavra

In [ ]:
!pip install googletrans==4.0.0rc1

In [ ]:
import requests
import easyocr
import cv2
from googletrans import Translator

# Carregando a imagem e criando o texto via OCR
imagem = cv2.imread('/content/geralt.jpg')
reader = easyocr.Reader(lang_list=['en'], gpu = True)
texto = ' '.join(reader.readtext(image = imagem, detail = 0))

# Inicializa o tradutor
translator = Translator()

# Traduz a palavra para o inglês
texto_en = translator.translate(texto, dest='pt').text

print(texto_en)